<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Riley/ETL_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-2.4.8'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Individual+Contributions+2013+(UF).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Individual+Contributions+2013+(UF).csv"), sep=",", header=True)
df.show()

In [ ]:
# Remove multiple columns
list = ['COMMITTEE', 'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY', 'EMPSTATE', 'PREVAMNT','REFUNDDATE', 'PAY_METHOD','INTERMNO', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df = df.drop(*list)
df.show()

In [ ]:
# check data types
df.dtypes

In [ ]:
#Change data types?

In [ ]:
#change the column names

df1 = df.withColumnRenamed("Employee ID", "employee_id") \
        .withColumnRenamed("Email", "email") \
        .withColumnRenamed("Gender", "gender") \
        .withColumnRenamed("Hire Date", "hire_date") \
        .withColumnRenamed("DOB", "dob") \
        .withColumnRenamed("Encrypted Password", "password")
df1.show(5)

In [ ]:
#Drop/Remove all refunds (<=0) from AMNT column


In [ ]:
#Possibly better code for below outcome
#Call only Mayor (1) and particaptes (P) within Dataframe
#df3 = df1.filter((df2.OFFICECD=="1") & (df2.CANCLASS=="P"))
#df3.show()


In [ ]:
#Check the above dataframe to make sure only unbique values between the OFFICECD is 1 
df3.select('OFFICECD').distinct().show

In [ ]:
#Check the CANCLASS is only P
df3.select('CANCLASS').distinct().show

In [ ]:
# ADD AMNT and MATCHAMNT


df = df.withColumn("TOTAL_AMNT", col("AMNT")+col("MATCHAMNT"))
df.show()

In [ ]:
#Create dataFrame based off Candidate ID and sum of AMT + MTCHAMNT

In [ ]:
#Filter date for 4 years previous of election (EX 2017 election - Date between 01-01-2013:12-31-2017)